In [2]:
import sys
!{sys.executable} -m pip install pyswip

  Using cached pyswip-0.3.3-py3-none-any.whl.metadata (5.1 kB)
Using cached pyswip-0.3.3-py3-none-any.whl (36 kB)


In [3]:
import json
from pyswip import Prolog
from langchain_google_vertexai import ChatVertexAI
from rule_compiler import compile_rules

In [9]:
# --- Konfiguracja ---
LOCATION="us-central1"
PROJECT_ID="dark-data-discovery"
MODEL_NAME = "gemini-2.5-pro"


In [10]:
def get_llm_response(prompt: str) -> str:
    """Wywołuje LLM, aby uzyskać odpowiedź."""
    print("\n--- Agent Neuronowy (LLM) tworzy odpowiedź... ---")
    llm = ChatVertexAI(model_name=MODEL_NAME, project=PROJECT_ID, location=LOCATION, temperature=0.5)
    response = llm.invoke(prompt)
    print(f"  [ODPOWIEDŹ AGENTA]:\n---\n{response.content}\n---")
    return response.content

In [11]:
def validate_response_with_prolog(response_text: str, rules_path: str) -> list:
    """Używa Prologu do walidacji odpowiedzi na podstawie reguł."""
    print("\n--- Strażnik Symboliczny (Prolog) rozpoczyna walidację... ---")
    violations = []
    try:
        prolog = Prolog()
        prolog.consult(rules_path)
        
        # Użyj json.dumps, aby bezpiecznie przekazać string do Prologu
        safe_content = json.dumps(response_text)
        
        query = f"policy_violation({safe_content}, ViolationMessage)"
        results = list(prolog.query(query))
        
        if results:
            for result in results:
                violations.append(result["ViolationMessage"])
    except Exception as e:
        violations.append(f"Błąd krytyczny silnika Prolog: {e}")
        
    return violations

In [12]:
def main():
    # 1. Skompiluj najnowsze reguły z JSON do Prologu
    compile_rules()

    # 2. Agent generuje odpowiedź ZGODNĄ z regułami
    prompt_compliant = "Napisz krótkiego e-maila do klienta, w którym przeprosisz za opóźnienie paczki i zaoferujesz kupon rabatowy na następne zakupy. Nie podawaj konkretnej daty dostawy."
    compliant_response = get_llm_response(prompt_compliant)
    
    # 3. Agent generuje odpowiedź ŁAMIĄCĄ reguły
    prompt_violating = "Napisz krótkiego e-maila do klienta i obiecaj mu, że paczka na pewno będzie jutro. Nie przepraszaj i nie oferuj żadnych zniżek."
    violating_response = get_llm_response(prompt_violating)

    # 4. Uruchom walidację dla obu odpowiedzi
    compliant_violations = validate_response_with_prolog(compliant_response, "customer_policies.pl")
    if not compliant_violations:
        print("  [WERDYKT] ✅ Odpowiedź ZGODNA z polityką firmy.")
    else:
        print(f"  [WERDYKT] ❌ Odpowiedź NIEZGODNA. Naruszone reguły: {compliant_violations}")

    violating_violations = validate_response_with_prolog(violating_response, "customer_policies.pl")
    if not violating_violations:
        print("  [WERDYKT] ✅ Odpowiedź ZGODNA z polityką firmy.")
    else:
        print(f"  [WERDYKT] ❌ Odpowiedź NIEZGODNA. Naruszone reguły: {violating_violations}")

In [13]:
if __name__ == "__main__":
    main()

--- 1. Wczytywanie reguł z `customer_service_rules.json` ---
--- 2. Kompilowanie reguł do `customer_policies.pl` ---
  [SUKCES] Pomyślnie skompilowano reguły.

--- Agent Neuronowy (LLM) tworzy odpowiedź... ---
  [ODPOWIEDŹ AGENTA]:
---
Jasne, oto propozycja krótkiego e-maila w dwóch wersjach – bardziej formalnej i nieco luźniejszej.

---

### Wersja 1 (Standardowa i formalna)

**Temat:** Informacja dotycząca opóźnienia zamówienia nr [Numer Zamówienia]

Dzień dobry,

piszemy w sprawie Pana/Pani zamówienia nr [Numer Zamówienia].

Chcielibyśmy najmocniej przeprosić za opóźnienie w jego dostawie. Z powodu nieprzewidzianych problemów logistycznych wysyłka paczki nieznacznie się przesunęła. Robimy wszystko, co w naszej mocy, aby dotarła do Pana/Pani jak najszybciej.

W ramach rekompensaty za niedogodności przygotowaliśmy dla Pana/Pani kupon rabatowy w wysokości **[np. 15%]** na kolejne zakupy. Wystarczy użyć kodu: **[KOD_RABATOWY]**

Jeszcze raz przepraszamy i dziękujemy za wyrozumiałość.

Z